In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -q -U kagglehub --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.3/731.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""
import keras
import keras_nlp
import kagglehub


# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("kaggle_username")
# os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_key")

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

In [ ]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/kaggle-docs/questions_answers"
    preset ="gpt2_base_en"# "gemma2_2b_en" # name of pretrained Gemma 2
    sequence_length = 512 # max size of input sequence for training
    batch_size = 10 # size of the input batch in training
    lora_rank = 4 # rank for LoRA, higher means more trainable parameters
    learning_rate=3e-4 # learning rate used in train
    epochs = 10 # number of epochs to train

In [ ]:
keras.utils.set_random_seed(Config.seed)


In [ ]:
pdf=df.iloc[1060]
pdf

NameError: name 'df' is not defined

In [ ]:
def filter_non_string_rows(df):
    df = df[df.iloc[:, 5].apply(lambda x: isinstance(x, str))]
    return df

In [ ]:
df = pd.read_csv("/content/MERGED_SONG_LYRIC_DATA.csv")

df = filter_non_string_rows(df)

df.head()

,Unnamed: 0.1,Artist,Title,Album,Date,Lyric,Year,Unnamed: 0
0,0,Ariana Grande,"​thank u, next","thank u, next",11/3/2018,thought i'd end up with sean but he wasn't a m...,2018.0,NaN
1,1,Ariana Grande,7 rings,"thank u, next",1/18/2019,yeah breakfast at tiffany's and bottles of bub...,2019.0,NaN
2,2,Ariana Grande,​God is a woman,Sweetener,7/13/2018,you you love it how i move you you love it how...,2018.0,NaN
3,3,Ariana Grande,Side To Side,Dangerous Woman,5/20/2016,ariana grande nicki minaj i've been here all ...,2016.0,NaN
4,4,Ariana Grande,​​no tears left to cry,Sweetener,4/20/2018,right now i'm in a state of mind i wanna be in...,2018.0,NaN


In [ ]:
df = df[df["Artist"] != "BTS (방탄소년단)"]


In [ ]:
template = "\n\nGive me a song in style of artist {Artist}\n\n\nSong:\n{Song}"
df["prompt"] = df.apply(lambda row: template.format(Artist=row.Artist,
                                                            #  Question=row.Question,
                                                             Song=row.Lyric), axis=1)
data = df.prompt.tolist()

In [ ]:
data_500

NameError: name 'data_500' is not defined

In [ ]:
data_500=data

In [ ]:
data_500_cx_200=[]

In [ ]:
for i in data_500:
    data_500_cx_200.append(i[:1500])

In [ ]:
data_500_cx_200

["\n\nGive me a song in style of artist Ariana Grande\n\n\nSong:\nthought i'd end up with sean but he wasn't a match wrote some songs about ricky now i listen and laugh even almost got married and for pete i'm so thankful wish i could say thank you to malcolm 'cause he was an angel  pre one taught me love one taught me patience and one taught me pain now i'm so amazing say i've loved and i've lost but that's not what i see so look what i got look what you taught me and for that i say   thank you next next thank you next next thank you next i'm so fuckin' grateful for my ex thank you next next thank you next next thank you next next i'm so fuckin'   spend more time with my friends i ain't worried 'bout nothin' plus i met someone else we havin' better discussions i know they say i move on too fast but this one gon' last 'cause her name is ari and i'm so good with that so good with that  pre she taught me love love she taught me patience patience how she handles pain pain that shit's amaz

In [ ]:
len(df)

5711

In [ ]:
print(df.iloc[0]["prompt"])



Give me a song in style of artist Ariana Grande


Song:
thought i'd end up with sean but he wasn't a match wrote some songs about ricky now i listen and laugh even almost got married and for pete i'm so thankful wish i could say thank you to malcolm 'cause he was an angel  pre one taught me love one taught me patience and one taught me pain now i'm so amazing say i've loved and i've lost but that's not what i see so look what i got look what you taught me and for that i say   thank you next next thank you next next thank you next i'm so fuckin' grateful for my ex thank you next next thank you next next thank you next next i'm so fuckin'   spend more time with my friends i ain't worried 'bout nothin' plus i met someone else we havin' better discussions i know they say i move on too fast but this one gon' last 'cause her name is ari and i'm so good with that so good with that  pre she taught me love love she taught me patience patience how she handles pain pain that shit's amazing yeah

In [ ]:
def colorize_text(text):
    for word, color in zip(["Artist",  "Song"], ["blue",  "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
password:733ca2e5335e338ca6d13e1b79891905
uname: malaydamani

SyntaxError: invalid decimal literal (<ipython-input-22-afa2434b7c85>, line 1)

In [ ]:
gemma_causal_lm = keras_nlp.models.GPT2CausalLM.from_preset(Config.preset)
gemma_causal_lm.summary()

100%|██████████| 431/431 [00:00<00:00, 1.08MB/s]


100%|██████████| 475M/475M [00:09<00:00, 52.5MB/s]


100%|██████████| 618/618 [00:00<00:00, 1.14MB/s]


100%|██████████| 0.99M/0.99M [00:00<00:00, 2.71MB/s]


100%|██████████| 446k/446k [00:00<00:00, 1.53MB/s]


...

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_causal_lm = gemma_causal_lm

    def query(self, artist):
        response = self.gemma_causal_lm.generate(
            self.prompt.format(
                Artist=artist,
                # Question=question,
                Song=""),
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))

In [ ]:
# tokenizer = gemma_causal_lm.preprocessor.tokenizer  # Replace with your tokenizer
# token_lengths = [len(tokenizer.tokenize(seq)) for seq in data]
# import matplotlib.pyplot as plt
# plt.hist(token_lengths, bins=50)
# plt.show()

KeyboardInterrupt: 

In [ ]:
# p

,1061
Unnamed: 0.1,1061
Artist,BTS (방탄소년단)
Title,Interlude
Album,2 Cool 4 Skool
Date,2013-06-12
Lyric,NaN
Year,2013.0
Unnamed: 0,202.0


In [ ]:
from tqdm import tqdm

In [ ]:
len(df)

5711

In [ ]:
x=0
y=0

In [ ]:
p=0

In [ ]:
for i in range(len(df)):
  p=df.iloc[i]
  # if isinstance(df.iloc[i,5],str):
  if len(df.iloc[i,5])>x:
      x=len(df.iloc[i,5])
      y=i
  # break

In [ ]:
p

,6026
Unnamed: 0.1,6026
Artist,Taylor Swift
Title,Find you
Album,NaN
Date,NaN
Lyric,trying just like they say just taking the step...
Year,NaN
Unnamed: 0,478.0
prompt,\n\nGive me a song in style of artist Taylor S...


In [ ]:
x

28673

In [ ]:
y

344

In [ ]:
len(tokenizer.tokenize(df.iloc[y,5]))

NameError: name 'tokenizer' is not defined

In [ ]:
# token_lengths = [len(tokenizer.tokenize(seq)) for seq in tqdm(data)]

  6%|▌         | 361/6027 [04:41<1:47:57,  1.14s/it]

KeyboardInterrupt: 

In [ ]:
# from joblib import Parallel, delayed
# from tqdm import tqdm

# token_lengths = Parallel(n_jobs=-1)(
#     delayed(lambda seq: len(tokenizer.tokenize(seq)))(seq) for seq in tqdm(data)
# )



  0%|          | 0/6027 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()

  6%|▌         | 361/6027 [07:22<1:55:48,  1.23s/it]

  1%|          | 32/6027 [00:45<1:58:12,  1.18s/it]/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
len(data_500_cx_200[0])

1500

In [ ]:
data_500_cx_200

["\n\nGive me a song in style of artist Ariana Grande\n\n\nSong:\nthought i'd end up with sean but he wasn't a match wrote some songs about ricky now i listen and laugh even almost got married and for pete i'm so thankful wish i could say thank you to malcolm 'cause he was an angel  pre one taught me love one taught me patience and one taught me pain now i'm so amazing say i've loved and i've lost but that's not what i see so look what i got look what you taught me and for that i say   thank you next next thank you next next thank you next i'm so fuckin' grateful for my ex thank you next next thank you next next thank you next next i'm so fuckin'   spend more time with my friends i ain't worried 'bout nothin' plus i met someone else we havin' better discussions i know they say i move on too fast but this one gon' last 'cause her name is ari and i'm so good with that so good with that  pre she taught me love love she taught me patience patience how she handles pain pain that shit's amaz

In [ ]:
#set sequence length cf. config (512)
gemma_causal_lm.preprocessor.sequence_length = Config.sequence_length

# Compile the model with loss, optimizer, and metric
gemma_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_causal_lm.fit(data_500_cx_200, epochs=15,
                    # Config.epochs,
                    batch_size=4
                    # Config.batch_size
                    )

Epoch 1/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 756s 510ms/step - loss: 1.7443 - sparse_categorical_accuracy: 0.4774
Epoch 2/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 708s 482ms/step - loss: 1.4672 - sparse_categorical_accuracy: 0.5324
Epoch 3/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 689s 481ms/step - loss: 1.2744 - sparse_categorical_accuracy: 0.5757
Epoch 4/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 689s 481ms/step - loss: 1.0969 - sparse_categorical_accuracy: 0.6179
Epoch 5/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 689s 481ms/step - loss: 0.9491 - sparse_categorical_accuracy: 0.6562
Epoch 6/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 689s 481ms/step - loss: 0.8119 - sparse_categorical_accuracy: 0.6931
Epoch 7/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 744s 519ms/step - loss: 0.6999 - sparse_categorical_accuracy: 0.7251
Epoch 8/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 690s 481ms/step - loss: 0.6034 - sparse_categorical_accuracy: 0.7534
Epoch 9/15
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 689s 481ms/step - loss: 0.5188 - sparse_categorical_accuracy: 0.7801
E

In [ ]:
gemma_qa = GemmaQA()


In [ ]:
row = df.iloc[0]
gemma_qa.query("Ariana Grande")



Give me a song in style of artist Ariana Grande


**<font color='green'>Song:</font>**
you're so gorgeous you give it to me so bad that i don't even know you at all you look so gorgeous in the bedroom and when we talk you make me laugh um i love you and i'm so sick of you  pre 'cause you're so gorgeous when you touch me and you make my heart go round   cause every thing that you do you make me wanna give it all away like i'm in your car and you're the one who's my girl and you're the one who's my lover   you're so gorgeous i can't say anything to your face 'cause look at your face gorgeous and i don't feel so bad when i look at you you make my head spin every time i look at you and i don't feel so bad  pre 'cause you're so gorgeous when you touch me and you make my heart go round   you're so gorgeous i can't say anything to your face 'cause look at your face look at your face gorgeous and i don't feel so bad cause look at your face look at your face look at your face   your eyes are so good your hands are so soft and your laugh is really something i can't ignore i think you look at my face and i don't feel so bad  pre 'cause you're so gorgeous when you touch me and you make my heart go round   cause every thing that you do you make me wanna give it all away like i'm in your car and you're the one who's my girl and you're the one who's my lover your eyes are so gorgeous look at your face look at your face look at your face look at your face   look at your face oh look at your face look at your han

In [ ]:
df["Artist"].unique()

array(['Ariana Grande', 'Beyoncé', 'Billie Eilish', 'Cardi B',
       'Charlie Puth', 'Coldplay', 'Drake', 'Dua Lipa', 'Ed Sheeran',
       'Eminem', 'Justin Bieber', 'Katy Perry', 'Khalid', 'Lady Gaga',
       'Maroon 5', 'Nicki Minaj', 'Post Malone', 'Rihanna',
       'Selena Gomez', 'Taylor Swift'], dtype=object)

In [ ]:
gemma_causal_lm.save("gpt_2_kaggle_data.keras")
from google.colab import drive
drive.mount('/content/drive')
import os
!cp -r /content/gpt_2_kaggle_data.keras /content/drive/MyDrive/gpt_2_kaggle_data.keras
gemma_causal_lm.save("gpt_2_kaggle_data.keras")

In [ ]:
gemma_causal_lm.save("gpt_2_kaggle_data.keras")
from google.colab import drive
drive.mount('/content/drive')
import os
!cp -r /content/gpt_2_kaggle_data.keras /content/drive/MyDrive/gpt_2_kaggle_data.keras
gemma_causal_lm.save("gpt_2_kaggle_data.keras")

Mounted at /content/drive
